In [5]:
# === ЛР2: Исследования модели (Heart Disease Classification) ===
import pandas as pd
import numpy as np
import os
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

print("✅ Импорт библиотек выполнен успешно")


✅ Импорт библиотек выполнен успешно


In [6]:
# === Пути и загрузка данных ===
DATA_PATH = "../data/clean_dataset.pkl"  # путь к очищенным данным из ЛР1
TARGET_COL = "target"
task_type = "classification"

if os.path.exists(DATA_PATH):
    df = pd.read_pickle(DATA_PATH)
else:
    raise FileNotFoundError(f"Файл {DATA_PATH} не найден. Проверь путь к датасету.")

print("✅ Данные загружены. Размер:", df.shape)
print("Столбцы:", df.columns.tolist())

# Проверка наличия целевой переменной
assert TARGET_COL in df.columns, f"Колонка {TARGET_COL} не найдена"

# Разделение на X и y
y = df[TARGET_COL]
X = df.drop(columns=[TARGET_COL])


✅ Данные загружены. Размер: (303, 14)
Столбцы: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']


In [7]:
# === Деление на train/test ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

print(f"Размер train: {X_train.shape}, test: {X_test.shape}")

# === Определим типы признаков ===
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

print("Числовые признаки:", numeric_features)
print("Категориальные признаки:", categorical_features)


Размер train: (227, 13), test: (76, 13)
Числовые признаки: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
Категориальные признаки: []


In [9]:
#%% 4. Baseline pipeline (классификация) + метрики

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Трансформации признаков
transform_baseline = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", TargetEncoder(), categorical_features),
    ],
    remainder="drop"
)

# Модель
baseline_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

# Пайплайн
pipe_baseline = Pipeline([
    ("transform", transform_baseline),
    ("classification", baseline_model)
])

# Обучение
pipe_baseline.fit(X_train, y_train)

# Предсказания
y_pred = pipe_baseline.predict(X_test)
y_pred_proba = pipe_baseline.predict_proba(X_test)[:, 1]

# Метрики baseline-модели
metrics_baseline = {
    "precision": precision_score(y_test, y_pred),
    "recall": recall_score(y_test, y_pred),
    "f1": f1_score(y_test, y_pred),
    "roc_auc": roc_auc_score(y_test, y_pred_proba)
}

metrics_baseline


{'precision': 0.7777777777777778,
 'recall': 0.8536585365853658,
 'f1': 0.813953488372093,
 'roc_auc': 0.8794425087108014}

In [11]:
#%% 5. Расширение признаков средствами sklearn + сохранение имён столбцов
from sklearn.preprocessing import PolynomialFeatures, KBinsDiscretizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from pathlib import Path

# Заменяем старые num_cols / cat_cols на актуальные
num_cols = numeric_features
cat_cols = categorical_features

# Выберем 2-3 признака под Polynomial и 2-3 под KBins
poly_cols = num_cols[:min(3, max(0, len(num_cols)))]
bin_cols = num_cols[min(3, len(num_cols)):min(6, len(num_cols))]
num_pass_cols = [c for c in num_cols if c not in set(poly_cols + bin_cols)]

num_transformers = []
if poly_cols:
    num_transformers.append(
        ("poly2_scale", Pipeline([
            ("scaler", StandardScaler()),
            ("poly2", PolynomialFeatures(degree=2, include_bias=False))
        ]), poly_cols)
    )
if bin_cols:
    num_transformers.append(
        ("kbins", KBinsDiscretizer(n_bins=5, encode="onehot-dense", strategy="quantile"), bin_cols)
    )
if num_pass_cols:
    num_transformers.append(("num_rest_scale", StandardScaler(), num_pass_cols))

transform_fe_sklearn = ColumnTransformer(
    transformers=[
        *num_transformers,
        ("cat", TargetEncoder(), cat_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=True
)

Xt_train_fe = transform_fe_sklearn.fit_transform(X_train, y_train)
feat_names = transform_fe_sklearn.get_feature_names_out()
print("Число признаков после FE:", Xt_train_fe.shape[1])

# Сохраняем имена признаков
PROJ_DIR = Path.cwd().parent
RESEARCH_DIR = PROJ_DIR / "research"
RESEARCH_DIR.mkdir(exist_ok=True)
FE_SKLEARN_COLS_PATH = RESEARCH_DIR / "fe_sklearn_columns.txt"

with open(FE_SKLEARN_COLS_PATH, "w", encoding="utf-8") as f:
    for c in feat_names:
        f.write(str(c) + "\n")

# Обучаем модель на новых признаках
rf_fe = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
pipe_fe = Pipeline([
    ("transform", transform_fe_sklearn),
    ("classification", rf_fe)
])

pipe_fe.fit(X_train, y_train)
y_pred_fe = pipe_fe.predict(X_test)
y_pred_proba_fe = pipe_fe.predict_proba(X_test)[:, 1]

metrics_fe = {
    "precision": precision_score(y_test, y_pred_fe),
    "recall": recall_score(y_test, y_pred_fe),
    "f1": f1_score(y_test, y_pred_fe),
    "roc_auc": roc_auc_score(y_test, y_pred_proba_fe)
}
metrics_fe


Число признаков после FE: 27


{'precision': 0.7659574468085106,
 'recall': 0.8780487804878049,
 'f1': 0.8181818181818182,
 'roc_auc': 0.8923344947735191}

In [12]:
#%% 6. Логирование модели с расширенными признаками (sklearn FE)
RUN_NAME = "fe_sklearn"

with mlflow.start_run(run_name=RUN_NAME) as run:
    mlflow.log_metrics(metrics_fe)
    mlflow.log_params({
        "fe_type": "poly2 + kbins",
        "n_estimators": rf_fe.n_estimators
    })
    mlflow.log_artifact(str(FE_SKLEARN_COLS_PATH), artifact_path="artifacts")

    input_example = X_train.head(5)
    signature = infer_signature(model_input=X_train.head(5), model_output=pipe_fe.predict(X_train.head(5)))
    mlflow.sklearn.log_model(
        pipe_fe, "model",
        signature=signature,
        input_example=input_example
    )
    print("run_id:", run.info.run_id)


2025/10/29 23:37:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run fe_sklearn at: http://127.0.0.1:5000/#/experiments/1/runs/c9512ad74d4c45e0a87413082438b060.
2025/10/29 23:37:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


run_id: c9512ad74d4c45e0a87413082438b060


In [14]:
#%% 7. SFS (forward) на FE-признаках через scikit-learn (совместимо со sklearn>=1.6)
import numpy as np
from sklearn.feature_selection import SequentialFeatureSelector as SKSFS
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

# Признаки после трансформаций FE
Xt_train_full = transform_fe_sklearn.transform(X_train)
n_feats = Xt_train_full.shape[1]
k = max(1, int(0.5 * n_feats))  # возьмём ~50% (допустимый диапазон 20–70%)
print(f"Всего признаков: {n_feats}; выбираем: {k}")

est = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

sfs = SKSFS(
    est,
    n_features_to_select=k,
    direction="forward",
    scoring="f1",   # метрика для классификации
    cv=3,
    n_jobs=-1
)
sfs.fit(Xt_train_full, y_train)

# Получим индексы и имена выбранных фич
support_mask = sfs.get_support()
selected_idx = np.where(support_mask)[0].tolist()
selected_names = [feat_names[i] for i in selected_idx]
print("Первые 10 индексов:", selected_idx[:10])

# Сохраним имена и индексы в файлы
SFS_IDX_PATH = RESEARCH_DIR / "sfs_indices.json"
SFS_NAMES_PATH = RESEARCH_DIR / "sfs_feature_names.txt"
json.dump(selected_idx, open(SFS_IDX_PATH, "w"))
with open(SFS_NAMES_PATH, "w", encoding="utf-8") as f:
    for n in selected_names:
        f.write(str(n) + "\n")

# Трансформер, выбирающий нужные столбцы по индексам
class ColumnIndexSelector(BaseEstimator, TransformerMixin):
    def __init__(self, indices):
        self.indices = np.array(indices, dtype=int)
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[:, self.indices]

# Пайплайн с выбором признаков
from sklearn.pipeline import Pipeline
pipe_sfs = Pipeline([
    ("transform", transform_fe_sklearn),
    ("select", ColumnIndexSelector(selected_idx)),
    ("classification", RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1))
])

# Обучение и метрики
pipe_sfs.fit(X_train, y_train)
y_pred_sfs = pipe_sfs.predict(X_test)
y_proba_sfs = pipe_sfs.predict_proba(X_test)[:, 1]

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
metrics_sfs = {
    "precision": precision_score(y_test, y_pred_sfs),
    "recall": recall_score(y_test, y_pred_sfs),
    "f1": f1_score(y_test, y_pred_sfs),
    "roc_auc": roc_auc_score(y_test, y_proba_sfs),
}
metrics_sfs


Всего признаков: 27; выбираем: 13
Первые 10 индексов: [6, 7, 8, 11, 12, 14, 16, 18, 19, 22]


{'precision': 0.7619047619047619,
 'recall': 0.7804878048780488,
 'f1': 0.7710843373493976,
 'roc_auc': 0.8634146341463415}

In [15]:
#%% 8. Логирование модели с SFS-отбором (классификация)
from mlflow.models import infer_signature
import mlflow

RUN_NAME = "fe_sklearn + sfs_forward (sklearn SFS)"

with mlflow.start_run(run_name=RUN_NAME) as run:
    mlflow.log_metrics(metrics_sfs)
    mlflow.log_params({
        "sfs_impl": "sklearn",
        "direction": "forward",
        "k_features": len(selected_idx),
        "n_all_features": n_feats
    })
    mlflow.log_artifact(str(FE_SKLEARN_COLS_PATH), artifact_path="artifacts")
    mlflow.log_artifact(str(SFS_IDX_PATH), artifact_path="artifacts")
    mlflow.log_artifact(str(SFS_NAMES_PATH), artifact_path="artifacts")

    input_example = X_train.head(5)
    signature = infer_signature(model_input=X_train.head(5), model_output=pipe_sfs.predict(X_train.head(5)))
    mlflow.sklearn.log_model(pipe_sfs, "model", signature=signature, input_example=input_example)
    print("run_id:", run.info.run_id)


run_id: ed836675659c44d2a43fa97a573ab12e


2025/10/29 23:49:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run fe_sklearn + sfs_forward (sklearn SFS) at: http://127.0.0.1:5000/#/experiments/1/runs/ed836675659c44d2a43fa97a573ab12e.
2025/10/29 23:49:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


In [18]:
#%% Патч: совместимый с sklearn.clone селектор колонок
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class ColumnIndexSelector(BaseEstimator, TransformerMixin):
    def __init__(self, indices):
        # НИЧЕГО не преобразуем здесь — оставляем ровно то, что передали (list)
        self.indices = indices

    def fit(self, X, y=None):
        # Преобразуем в массив только во время fit, записывая в служебный атрибут
        self._indices_array_ = np.asarray(self.indices, dtype=int)
        return self

    def transform(self, X):
        return X[:, self._indices_array_]


In [19]:
#%% 9. Optuna: подбор гиперпараметров (классификация)
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold

def build_model(params, selected_idx_):
    # превращаем индексы в обычный Python-список (clone не умеет работать с numpy.array)
    selector = ColumnIndexSelector(indices=list(selected_idx_))
    rf = RandomForestClassifier(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        max_features=params["max_features"],
        random_state=42,
        n_jobs=-1
    )
    return Pipeline([
        ("transform", transform_fe_sklearn),
        ("select", selector),
        ("classification", rf)
    ])

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 600),
        "max_depth": trial.suggest_int("max_depth", 4, 40),
        "max_features": trial.suggest_float("max_features", 0.1, 1.0)
    }
    pipe = build_model(params, selected_idx)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    # используем F1 как метрику качества (maximize)
    scores = cross_val_score(pipe, X_train, y_train, cv=cv, scoring="f1", n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=12, show_progress_bar=True)

best_params = study.best_params
best_score = study.best_value
best_params, best_score


[I 2025-10-29 23:53:26,732] A new study created in memory with name: no-name-aaf724e8-88ff-466c-903d-c4f51bc2509d
Best trial: 0. Best value: 0.837257:   8%|▊         | 1/12 [00:05<01:03,  5.79s/it]

[I 2025-10-29 23:53:32,531] Trial 0 finished with value: 0.8372567180623407 and parameters: {'n_estimators': 247, 'max_depth': 8, 'max_features': 0.13571127222833912}. Best is trial 0 with value: 0.8372567180623407.


Best trial: 1. Best value: 0.841973:  17%|█▋        | 2/12 [00:08<00:41,  4.11s/it]

[I 2025-10-29 23:53:35,450] Trial 1 finished with value: 0.8419730761502914 and parameters: {'n_estimators': 334, 'max_depth': 27, 'max_features': 0.880997188339568}. Best is trial 1 with value: 0.8419730761502914.


Best trial: 1. Best value: 0.841973:  25%|██▌       | 3/12 [00:12<00:35,  3.99s/it]

...

Best trial: 6. Best value: 0.855472:  92%|█████████▏| 11/12 [00:36<00:03,  3.01s/it]

[I 2025-10-29 23:54:02,887] Trial 10 finished with value: 0.84721789195818 and parameters: {'n_estimators': 178, 'max_depth': 40, 'max_features': 0.5891833945469519}. Best is trial 6 with value: 0.8554724990771502.


Best trial: 6. Best value: 0.855472: 100%|██████████| 12/12 [00:37<00:00,  3.10s/it]

[I 2025-10-29 23:54:03,943] Trial 11 finished with value: 0.8381199438358417 and parameters: {'n_estimators': 129, 'max_depth': 30, 'max_features': 0.5715619619019107}. Best is trial 6 with value: 0.8554724990771502.


({'n_estimators': 224, 'max_depth': 35, 'max_features': 0.5808428712630324},
 0.8554724990771502)

In [21]:
#%% 10. Обучение лучшей модели и логирование (+ регистрация версии)
pipe_best = build_model(best_params, selected_idx)
pipe_best.fit(X_train, y_train)
y_pred_best = pipe_best.predict(X_test)

metrics_best = {
    "precision": precision_score(y_test, y_pred_best),
    "recall": recall_score(y_test, y_pred_best),
    "f1": f1_score(y_test, y_pred_best),
    "roc_auc": roc_auc_score(y_test, pipe_best.predict_proba(X_test)[:, 1]),
}

RUN_NAME = "optuna_tuned_rf"

with mlflow.start_run(run_name=RUN_NAME) as run:
    mlflow.log_metrics(metrics_best)
    mlflow.log_params(best_params)
    mlflow.log_artifact(str(FE_SKLEARN_COLS_PATH), artifact_path="artifacts")
    mlflow.log_artifact(str(SFS_IDX_PATH), artifact_path="artifacts")
    mlflow.log_artifact(str(SFS_NAMES_PATH), artifact_path="artifacts")

    input_example = X_train.head(5)
    signature = infer_signature(
        model_input=X_train.head(5),
        model_output=pipe_best.predict(X_train.head(5))
    )
    mlflow.sklearn.log_model(
        pipe_best,
        "model",
        signature=signature,
        input_example=input_example,
        registered_model_name=MODEL_NAME
    )

    print("run_id:", run.info.run_id)


Successfully registered model 'estate_rf_regressor'.
2025/10/29 23:55:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: estate_rf_regressor, version 1
Created version '1' of model 'estate_rf_regressor'.
2025/10/29 23:55:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run optuna_tuned_rf at: http://127.0.0.1:5000/#/experiments/1/runs/af6164cda4984c9587c4ede2d95ed247.
2025/10/29 23:55:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


run_id: af6164cda4984c9587c4ede2d95ed247


In [23]:
#%% 11. Финальный прогон: обучаем на всей выборке и ставим тег Production
pipe_prod = build_model(best_params, selected_idx)
pipe_prod.fit(X, y)

RUN_NAME = "production_fullfit"
with mlflow.start_run(run_name=RUN_NAME) as run:
    mlflow.log_artifact(str(FE_SKLEARN_COLS_PATH), artifact_path="artifacts")
    mlflow.log_artifact(str(SFS_IDX_PATH), artifact_path="artifacts")
    mlflow.log_artifact(str(SFS_NAMES_PATH), artifact_path="artifacts")

    input_example = X.head(5)
    signature = infer_signature(
        model_input=X.head(5),
        model_output=pipe_prod.predict(X.head(5))
    )
    mlflow.sklearn.log_model(
        pipe_prod,
        "model",
        signature=signature,
        input_example=input_example,
        registered_model_name=MODEL_NAME
    )
    prod_run_id = run.info.run_id
    print("run_id:", prod_run_id)

# Проставим тег "Production" на последней версии модели
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
versions = client.get_latest_versions(MODEL_NAME)
latest = sorted(versions, key=lambda v: int(v.version))[-1]
client.set_model_version_tag(MODEL_NAME, latest.version, "stage", "Production")
print(f"MODEL '{MODEL_NAME}' version {latest.version} помечена тегом Production")


Registered model 'estate_rf_regressor' already exists. Creating a new version of this model...
2025/10/29 23:56:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: estate_rf_regressor, version 2
Created version '2' of model 'estate_rf_regressor'.
2025/10/29 23:56:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run production_fullfit at: http://127.0.0.1:5000/#/experiments/1/runs/3dd1099eae3040e286f0006a0666caa2.
2025/10/29 23:56:28 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1.


run_id: 3dd1099eae3040e286f0006a0666caa2
MODEL 'estate_rf_regressor' version 2 помечена тегом Production


In [26]:
#%% Скачать MLmodel по известному run_id (production_fullfit)
import os, mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

prod_run_id = "3dd1099eae3040e286f0006a0666caa2"   # <-- твой реальный run_id
dst = "research"
os.makedirs(dst, exist_ok=True)

path = mlflow.artifacts.download_artifacts(
    run_id=prod_run_id,
    artifact_path="model/MLmodel",
    dst_path=dst
)
print("Saved to:", path)


Saved to: /home/user/my_proj/research/research/model/MLmodel
